In [1]:
import pandas as pd
import gspread
import re
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from sqlalchemy import create_engine, text
import urllib.parse

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/documents']

credentials = Credentials.from_service_account_file('automations-388509-b668130c76a4.json', scopes=scopes)

drive_service = build('drive', 'v3', credentials=credentials)

gc = gspread.authorize(credentials)
gs = gc.open_by_key('1KLuXOEZc_7D27H_91aISpil87MGiTE8pEumdyhQRQo4')
ws = gs.worksheet('Copy of Final')
sheet_config = pd.DataFrame(ws.get_all_values()[1:], columns=ws.get_all_values()[0])  # Avoid calling pop(0)
sheet_config = sheet_config[sheet_config['active'] == 'Y']
tasks = sheet_config['task'].unique()

# Set database connection details
user = 'admin'
password = 'rr_1234567'
host = 'rr-raw.cybvhc8iyrsv.us-east-1.rds.amazonaws.com'
port = 3306
database = 'snabb_db'

# Create a database engine
password_encoded = urllib.parse.quote_plus(password)
db_url = f'mysql+mysqlconnector://{user}:{password_encoded}@{host}:{port}/{database}?connect_timeout=1000'
engine = create_engine(db_url)
conn = engine.connect()

def process_task(task, task_data, engine, drive_service, gc):
    doc_id, gs_id, worksheet_name, should_clear = task_data.iloc[0][1:5]

    # Retrieve document content
    doc = drive_service.files().export_media(fileId=doc_id, mimeType='text/plain').execute()
    query = re.sub(r'[^\x20-\x7E]+', '', doc.decode('utf-8')).strip()
    query = str(query)

    # Fetch the query result and create a DataFrame
    print(f"Executing query: {query}")
    query_text = text(query)
    df = pd.read_sql(query_text, conn)
    print(df)
    gs = gc.open_by_key(gs_id)
    worksheet = gs.worksheet(worksheet_name)

    if should_clear == 'Y':
        worksheet.clear()

    # Write DataFrame to worksheet
    set_with_dataframe(
        worksheet=worksheet,
        dataframe=df,
        include_index=False,
        include_column_header=True,
        resize=True)

    print(task)
    del df

for task in tasks:
    d = sheet_config[sheet_config['task'] == task]
    process_task(task, d, engine, drive_service, gc)

Executing query: SELECT     snabb_db.snabb_order_table.*,    CASE        WHEN `Primary Key` IS NOT NULL THEN DATE_FORMAT(CONCAT(YEAR(`Bill Date`), '-', LPAD(MONTH(`Bill Date`), 2, '0'), '-01'),'%d/%m/%Y')    END AS `Month`,    CASE WHEN `Primary Key` IS NOT NULL THEN LEFT(snabb_order_table.`Channel`, 4) END AS Brand,    (SELECT         SUM(snabb_margin_table.Margin * snabb_order_table.`Item Subtotal`)     FROM         snabb_margin_table     WHERE         snabb_margin_table.Item_name = snabb_order_table.`Item Name`        AND snabb_margin_table.Outlet = snabb_order_table.Outlet        AND snabb_margin_table.Start_date <= snabb_order_table.`Bill Date`        AND snabb_margin_table.End_date >= snabb_order_table.`Bill Date`    ) AS Margin,    CASE         WHEN TRIM(snabb_order_table.`Bill Date`) = '' THEN ' '        ELSE WEEK(STR_TO_DATE(snabb_order_table.`Bill Date`, '%Y-%m-%d'), 3)    END AS `week`,    COALESCE(snabb_key_table.`Final item name`, "N/A") AS products,    COALESCE(snabb_key_